In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn

2023-02-02 20:56:28.557011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Dropout, BatchNormalization, Flatten, Dense, Input, concatenate

# Data Handling

In [3]:
y_train_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/Classification/y_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')
y_dev_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/Classification/y_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')

y_train_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/Classification/y_train_BoAW_openSMILE_230_MFCC.gz', delimiter=',')
y_dev_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/Classification/y_dev_BoAW_openSMILE_230_MFCC.gz', delimiter=',')

In [4]:
# BoVW_openFace_210_Pose_Gaze_AUs
# Data Handling

x_train_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/x_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(163, 19663, 100)
x_dev_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/x_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(56, 36277, 100)

x_train_bovw_input_shape = x_train_bovw.shape
x_dev_bovw_input_shape = x_dev_bovw.shape

zeros = np.zeros((163, x_dev_bovw_input_shape[1]-x_train_bovw_input_shape[1], x_train_bovw_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_train_bovw and x_dev_bovw equal
x_train_bovw = np.concatenate((x_train_bovw, zeros), axis=1)

print(x_train_bovw.shape)
print(x_dev_bovw.shape)

(163, 36277, 100)
(56, 36277, 100)


In [5]:
# BoAW_openSMILE_230_MFCC
# Data Handling
x_train_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/x_train_BoAW_openSMILE_230_MFCC.gz', delimiter=',').reshape(163, 19579, 100)
x_dev_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/x_dev_BoAW_openSMILE_230_MFCC.gz', delimiter=',').reshape(56, 29679, 100)

x_train_boaw_input_shape = x_train_boaw.shape
x_dev_boaw_input_shape = x_dev_boaw.shape

zeros = np.zeros((163, x_dev_boaw_input_shape[1]-x_train_boaw_input_shape[1], x_train_boaw_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_train_boaw and x_dev_boaw equal
x_train_boaw = np.concatenate((x_train_boaw, zeros), axis=1)

print(x_train_boaw.shape)
print(x_dev_boaw.shape)

(163, 29679, 100)
(56, 29679, 100)


In [6]:
x_train_bovw_input_shape = x_train_bovw.shape
x_train_boaw_input_shape = x_train_boaw.shape

train_zeros = np.zeros((163, x_train_bovw_input_shape[1]-x_train_boaw_input_shape[1], x_train_bovw_input_shape[2]))
dev_zeros = np.zeros((56, x_dev_bovw_input_shape[1]-x_dev_boaw_input_shape[1], x_dev_bovw_input_shape[2]))


x_train_boaw = np.concatenate((x_train_boaw, train_zeros), axis=1)
x_dev_boaw = np.concatenate((x_dev_boaw, dev_zeros), axis=1)

print(x_train_boaw.shape)
print(x_dev_boaw.shape)

(163, 36277, 100)
(56, 36277, 100)


# CNN

In [7]:
# BoVW OpenFace
inputs_openFace = Input(shape=x_train_bovw.shape[1:])
c1 = Conv1D(100, (3), padding='same', activation='relu')(inputs_openFace)
bn1 = BatchNormalization()(c1)
d1 = Dropout(rate=0.8)(bn1)
mp1 = MaxPooling1D(pool_size=(5), padding='same')(d1)

c2 = Conv1D(64, (3), padding='same', activation='relu')(mp1)
bn2 = BatchNormalization()(c2)
d2 = Dropout(rate=0.5)(bn2)
mp2 = MaxPooling1D(pool_size=(5), padding='same')(d2)

c3 = Conv1D(32, (3), padding='same', activation='relu')(mp2)
bn3 = BatchNormalization()(c3)
d3 = Dropout(rate=0.5)(bn3)
mp3 = MaxPooling1D(pool_size=(5), padding='same')(d3)

c4 = Conv1D(16, (3), padding='same', activation='relu')(mp3)
bn4 = BatchNormalization()(c4)
d4 = Dropout(rate=0.5)(bn4)
flat_openFace = MaxPooling1D(pool_size=(5), padding='same')(d4)


model_openFace = Model(inputs=inputs_openFace, outputs=flat_openFace)
model_openFace.summary()

2023-02-02 21:01:07.885600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36277, 100)]      0         
                                                                 
 conv1d (Conv1D)             (None, 36277, 100)        30100     
                                                                 
 batch_normalization (BatchN  (None, 36277, 100)       400       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 36277, 100)        0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7256, 100)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 7256, 64)          19264 

In [8]:
# BoVW OpenSmile
inputs_openSmile = Input(shape=x_train_bovw.shape[1:])
c5 = Conv1D(100, (3), padding='same', activation='relu')(inputs_openSmile)
bn5 = BatchNormalization()(c5)
d5 = Dropout(rate=0.8)(bn5)
mp5 = MaxPooling1D(pool_size=(5), padding='same')(d5)

c6 = Conv1D(64, (3), padding='same', activation='relu')(mp5)
bn6 = BatchNormalization()(c6)
d6 = Dropout(rate=0.5)(bn6)
mp6 = MaxPooling1D(pool_size=(5), padding='same')(d6)

c7 = Conv1D(32, (3), padding='same', activation='relu')(mp6)
bn7 = BatchNormalization()(c7)
d7 = Dropout(rate=0.5)(bn7)
mp7 = MaxPooling1D(pool_size=(5), padding='same')(d7)

c8 = Conv1D(16, (3), padding='same', activation='relu')(mp7)
bn8 = BatchNormalization()(c8)
d8 = Dropout(rate=0.5)(bn8)
flat_openSmile = MaxPooling1D(pool_size=(5), padding='same')(d8)

model_openSmile = Model(inputs=inputs_openSmile, outputs=flat_openSmile)
model_openSmile.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 36277, 100)]      0         
                                                                 
 conv1d_4 (Conv1D)           (None, 36277, 100)        30100     
                                                                 
 batch_normalization_4 (Batc  (None, 36277, 100)       400       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 36277, 100)        0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 7256, 100)        0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 7256, 64)          1926

# Multi Modal (BoVW OpenFace and BoAW OpenSmile)

In [9]:
x = concatenate([flat_openFace, flat_openSmile], axis=1)

l1 = LSTM(16, return_sequences=True)(x)
bn9 = BatchNormalization()(l1)
d9 = Dropout(rate=0.5)(bn9)
mp9 = MaxPooling1D(pool_size=(5), padding='same')(d9)

l2 = LSTM(1, return_sequences=True)(mp9)
bn10 = BatchNormalization()(l2)
d10 = Dropout(rate=0.5)(bn10)

den1 = Dense(1, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), 
                    bias_regularizer=regularizers.L2(1e-4), 
                    activity_regularizer=regularizers.L2(1e-5), 
                    activation='relu')(d10)
d11 = Dropout(rate=0.2)(den1)

f1 = Flatten()(d11)
output = Dense(1, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), 
                    bias_regularizer=regularizers.L2(1e-4), 
                    activity_regularizer=regularizers.L2(1e-5), 
                    activation='sigmoid')(f1)

final_model = Model(inputs=[inputs_openFace, inputs_openSmile], outputs=[output])

final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss = 'binary_crossentropy', metrics= ['accuracy'])
final_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 36277, 100)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, 36277, 100)  0           []                               
                                ]                                                                 
                                                                                                  
 conv1d (Conv1D)                (None, 36277, 100)   30100       ['input_1[0][0]']                
                                                                                            

# Training and Validation Dataset

In [10]:
batch_size = 32

checkpoint_filepath = '/Users/suyashsingh/Documents/Projects/Depression_Analysis/checkpoints/multimodal_classification/weights-improvement-multimodal-hdf5-{epoch:02d}-{val_accuracy:.2f}.hdf5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy', 
    save_best_only=True,
    save_weights_only=True,
    mode='max'
    )

final_model.fit(x=[x_train_bovw, x_train_boaw],
                y=y_train_bovw,
                batch_size=batch_size,
                epochs=80,
                callbacks=[model_checkpoint_callback],
                validation_data=[[x_dev_bovw, x_dev_boaw], y_dev_bovw],
                shuffle=True,
                verbose=1
                )

Epoch 1/80
6/6 [==============================] - 130s 18s/step - loss: 0.7014 - accuracy: 0.4233 - val_loss: 0.6940 - val_accuracy: 0.2321
Epoch 2/80
6/6 [==============================] - 95s 15s/step - loss: 0.7089 - accuracy: 0.4294 - val_loss: 0.6936 - val_accuracy: 0.4464
Epoch 3/80
6/6 [==============================] - 66s 10s/step - loss: 0.7030 - accuracy: 0.4908 - val_loss: 0.6930 - val_accuracy: 0.5893
Epoch 4/80
6/6 [==============================] - 63s 10s/step - loss: 0.7074 - accuracy: 0.4417 - val_loss: 0.6923 - val_accuracy: 0.7500
Epoch 5/80
6/6 [==============================] - 62s 10s/step - loss: 0.7147 - accuracy: 0.4417 - val_loss: 0.6916 - val_accuracy: 0.7857
Epoch 6/80
6/6 [==============================] - 62s 10s/step - loss: 0.7185 - accuracy: 0.3926 - val_loss: 0.6909 - val_accuracy: 0.7857
Epoch 7/80
6/6 [==============================] - 64s 10s/step - loss: 0.7097 - accuracy: 0.3988 - val_loss: 0.6902 - val_accuracy: 0.7857
Epoch 8/80
6/6 [==========

In [ ]:
from keras.models import load_model
model = load_model('/Users/suyashsingh/Documents/Projects/Depression_Analysis/checkpoints/multimodal_classification/')

In [ ]:
x_test_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/x_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',').reshape(56, 16869, 100)
y_test_bovw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoVW/Classification/y_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', delimiter=',')

x_test_bovw_input_shape = x_test_bovw.shape
y_test_bovw_input_shape = y_test_bovw.shape

zeros_bovw = np.zeros((56, x_dev_bovw_input_shape[1]-x_test_bovw_input_shape[1], x_test_bovw_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_test_bovw and x_dev equal
x_test_bovw = np.concatenate((x_test_bovw, zeros_bovw), axis=1)

x_test_bovw_input_shape = x_test_bovw.shape

print(x_test_bovw_input_shape)
print(y_test_bovw_input_shape)

(56, 36277, 100)
(56,)


In [ ]:
x_test_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/x_test_BoAW_openSMILE_230_MFCC.gz', delimiter=',').reshape(56, 16674, 100)
y_test_boaw = np.loadtxt('/Users/suyashsingh/Documents/Projects/Depression_Analysis/data/BoAW/Classification/y_test_BoAW_openSMILE_230_MFCC.gz', delimiter=',')

x_test_boaw_input_shape = x_test_boaw.shape
y_test_boaw_input_shape = y_test_boaw.shape

zeros_boaw = np.zeros((56, x_dev_bovw_input_shape[1]-x_test_boaw_input_shape[1], x_test_boaw_input_shape[2]))

# adding rows with value 0 to make the dimensions of x_test_boaw and x_dev equal
x_test_boaw = np.concatenate((x_test_boaw, zeros_boaw), axis=1)

x_test_boaw_input_shape = x_test_boaw.shape

print(x_test_boaw_input_shape)
print(y_test_boaw_input_shape)

(56, 36277, 100)
(56,)


In [ ]:
score = final_model.evaluate(x=[x_test_bovw, x_test_boaw], y=y_test_bovw, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6852575540542603
Test accuracy: 0.6964285969734192


In [ ]:
y_pred = final_model.predict(x=[x_test_bovw, x_test_boaw])
print(y_pred)
print(y_test_bovw)

2/2 [==============================] - 11s 1s/step
[[0.48921442]
 [0.48868152]
 [0.4891607 ]
 [0.4916366 ]
 [0.49022624]
 [0.4887184 ]
 [0.48925307]
 [0.48941568]
 [0.4893601 ]
 [0.48965356]
 [0.48967648]
 [0.49095806]
 [0.4887647 ]
 [0.4893425 ]
 [0.48891744]
 [0.48910555]
 [0.48888502]
 [0.4887632 ]
 [0.4890568 ]
 [0.48886707]
 [0.48897836]
 [0.48953474]
 [0.48972422]
 [0.48934114]
 [0.48965472]
 [0.4899695 ]
 [0.4892835 ]
 [0.49080852]
 [0.48962307]
 [0.48903063]
 [0.489894  ]
 [0.48903343]
 [0.4891791 ]
 [0.48868236]
 [0.48947564]
 [0.48872894]
 [0.4894623 ]
 [0.48914135]
 [0.48913994]
 [0.49005643]
 [0.48940971]
 [0.48975748]
 [0.48896483]
 [0.4899709 ]
 [0.4891355 ]
 [0.49024856]
 [0.49041328]
 [0.48898923]
 [0.48909026]
 [0.48943806]
 [0.48884046]
 [0.48899326]
 [0.49004114]
 [0.48966852]
 [0.49021357]
 [0.49034294]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.